## Example - testing kinetic diffusive heating operator

This notebook tests whether the implemented common_models.py diffusive heating operator for the $f_0$ harmonic performs as expected.

This is the v2.0.0 rewrite of the original test.


In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.common_models as cm
from RMK_support import varFromNode,node


### Context initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_kin_heating_test/")
rk.mpiContext = rmk.MPIContext(4)

### Grid setup

In [ ]:
xGrid = 0.1*np.ones(128) 
dv0 = 0.01
cv = 1.025
vGrid = [dv0]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 0
rk.grid = rmk.Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True,isPeriodic=True)
L = sum(xGrid)

### Variables

In [ ]:
nInit = 1.0*np.ones(rk.grid.numX())
TInit = 1
fInit = np.zeros([rk.grid.numX(),rk.grid.numH(),rk.grid.numV()])
for i in range(rk.grid.numX()):
    fInit[i,rk.grid.getH(0)-1,:] = (TInit*np.pi)**(-1.5) * nInit[i] * np.exp(-rk.grid.vGrid**2/TInit)

# Rescale distribution function to ensure that the numerical density moment agrees with the initial values
numerical_dens = rk.grid.velocityMoment(fInit,0,1)
for i in range(rk.grid.numX()):
    fInit[i,rk.grid.getH(0)-1,:] = nInit[i] *fInit[i,rk.grid.getH(0)-1,:]/numerical_dens[i]

f = rmk.Variable("f",rk.grid,data=fInit,isDistribution=True)
W = rmk.Variable("W",rk.grid,derivation=rk.textbook["energyMoment"],derivationArgs=["f"])
n = rmk.Variable("n",rk.grid,derivation=rk.textbook["densityMoment"],derivationArgs=["f"])
T =  varFromNode("T",rk.grid,node=(2/3)*node(W)/node(n))

rk.variables.add(f,W,n,T)

### Adding the heating

In [ ]:
heatingModel = rmk.Model("heating")

energyInjectionRate = 0.02
eProf = np.zeros(128)
eProf[54:74] = energyInjectionRate
heatingModel.ddt[f] += cm.diffusiveHeatingTerm(rk.grid,rk.norms,f,n,heatingProfile=rk.grid.profile(eProf,"X","H"))
rk.models.add(heatingModel)

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = rmk.BDEIntegrator("BDE",absTol=10.0,convergenceVars=[f])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=0.1,steps=integrationStep) 
rk.integrationScheme.setFixedNumTimesteps(100,10) 

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Kinetic diffusive heating test")

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset

In [ ]:
dataset

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

### Test distribution function distortion due to heating operator

In [ ]:
dataName = 'f'
xInd = 60 

times = dataset.coords['t'].data
f0_analytic=np.zeros((len(times),rk.grid.numV()))
temps = dataset["T"].data
dens = dataset["n"].data
for i in range(len(times)):
        f0_analytic[i,:] = (temps[i,xInd]*np.pi)**(-1.5) * dens[i,xInd] * np.exp(-rk.grid.vGrid**2/temps[i,xInd])
        
curveDict = {t: hv.Curve([(v**2,dataset[dataName].data[t,xInd,0,i]) for i,v in enumerate(dataset.coords["v"].data)]).opts(logy=True,ylim=(1e-20,1e0),xlim=(0,60))*
              hv.Curve([(v**2,f0_analytic[t,i]) for i,v in enumerate(dataset.coords["v"].data)]).opts(logy=True,ylim=(1e-20,1e0),xlim=(0,60))for t in range(len(times))}
kdims = [hv.Dimension(('t', 'Time'),unit=dataset.coords["t"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()

### Check spatial profile

In [ ]:
curveDict = {t: hv.Curve(dataset["W"][{"t":t}]) for t in range(len(times))}
kdims = [hv.Dimension(('t', 'Time'),unit=dataset.coords["t"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()

### Check heating amount 

The error is likely mostly due to velocity space integration

In [ ]:
heatingError = (dataset["W"].data[1,63]-dataset["W"].data[0,63] - 0.02)/0.02
heatingError
